<a href="https://colab.research.google.com/github/rhiats/Heart-Failure-Clinical-Records/blob/main/Heart_Failure_Clinical_Records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Heart Failure Clinical Records**

Replicate paper:

Chicco, D., & Jurman, G. (2020). Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone. BMC Medical Informatics and Decision Making, 20, 16. https://doi.org/10.1186/s12911-020-1023-5

Dataset:
Heart Failure Clinical Records [Dataset]. (2020). UCI Machine Learning Repository. https://doi.org/10.24432/C5Z89R.

In [1]:
!pip install ucimlrepo

In [3]:
import pandas as pd

**Load Data from UCI ML Repo**

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
heart_failure_clinical_records = fetch_ucirepo(id=519)

# data (as pandas dataframes)
X = heart_failure_clinical_records.data.features
y = heart_failure_clinical_records.data.targets

# metadata
print(heart_failure_clinical_records.metadata)

# variable information
print(heart_failure_clinical_records.variables)

{'uci_id': 519, 'name': 'Heart Failure Clinical Records', 'repository_url': 'https://archive.ics.uci.edu/dataset/519/heart+failure+clinical+records', 'data_url': 'https://archive.ics.uci.edu/static/public/519/data.csv', 'abstract': 'This dataset contains the medical records of 299 patients who had heart failure, collected during their follow-up period, where each patient profile has 13 clinical features.', 'area': 'Health and Medicine', 'tasks': ['Classification', 'Regression', 'Clustering'], 'characteristics': ['Multivariate'], 'num_instances': 299, 'num_features': 12, 'feature_types': ['Integer', 'Real'], 'demographics': ['Age', 'Sex'], 'target_col': ['death_event'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Mon Feb 26 2024', 'dataset_doi': '10.24432/C5Z89R', 'creators': [], 'intro_paper': {'ID': 286, 'type': 'NATIVE', 'title': 'Machine learning can predict survival of patients with heart failure f

In [25]:
df = pd.concat([X, y], axis=1)
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,death_event
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


**Continuous Features EDA**

In [11]:
continuous_features_df = X[['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']]
summary_continuous_full_sample_df = pd.DataFrame({
    'Median': continuous_features_df.median(),
    'Mean': round(continuous_features_df.mean(),2),
    'Standard Deviation': round(continuous_features_df.std(),2)
})

In [12]:
summary_continuous_full_sample_df

,Median,Mean,Standard Deviation
age,60.0,60.83,11.89
creatinine_phosphokinase,250.0,581.84,970.29
ejection_fraction,38.0,38.08,11.83
platelets,262000.0,263358.03,97804.24
serum_creatinine,1.1,1.39,1.03
serum_sodium,137.0,136.63,4.41
time,115.0,130.26,77.61


In [28]:
continuous_features_dead_df = df[['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']][df['death_event']==0]

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time
14,49.0,80,30,427000.0,1.0,138,12
20,65.0,52,25,276000.0,1.3,137,16
23,53.0,63,60,368000.0,0.8,135,22
33,50.0,159,30,302000.0,1.2,138,29
38,60.0,2656,30,305000.0,2.3,137,30
...,...,...,...,...,...,...,...
294,62.0,61,38,155000.0,1.1,143,270
295,55.0,1820,38,270000.0,1.2,139,271
296,45.0,2060,60,742000.0,0.8,138,278
297,45.0,2413,38,140000.0,1.4,140,280


In [20]:
summary_continuous_dead_df = pd.DataFrame({
    'Median': continuous_features_dead_df.median(),
    'Mean': round(continuous_features_dead_df.mean(),2),
    'Standard Deviation': round(continuous_features_dead_df.std(),2)
})

In [21]:
summary_continuous_dead_df

,Median,Mean,Standard Deviation
age,NaN,NaN,NaN
creatinine_phosphokinase,NaN,NaN,NaN
ejection_fraction,NaN,NaN,NaN
platelets,NaN,NaN,NaN
serum_creatinine,NaN,NaN,NaN
serum_sodium,NaN,NaN,NaN
time,NaN,NaN,NaN


**Categorical Features EDA**

**Mann–Whitney U test**

**Pearson correlation coefficient**

**Scatterplot comparing Serum Creatine v. Ejection Fraction**

**Survival Month**

**Survival Month v. Survival**

**Survival Prediction on all Clinical Features**



1. Random Forests
2. Decision Tree
3. Gradient Boosting
4. One Rule
5. Artificial Neural Network
6. Naive Bayes
7. SVM Radial
8. SVM Linear
9. K-nearest Neighbor

Found Random Forest performed the best.







**Feature Ranking**


1. RReliefF
2. Max-Min Parents and Children
3. Random Forest
4. One Rule
5. Recursive Partitioning and Regression Trees
6. Support Vector Machines with linear kernel
7. eXtreme Gradient Boosting

11 features (everything but Time)

Borda list: lower the score,the more important the feature

The authors found the top–two features to be serum
creatinine and ejection fraction. They used these 2 features in 3 classifiers:
Random Forests (RF),
Support Vector Machine with Gaussian Kernel (GSVM)
and eXtreme Gradient Boosting (XGB).


**Random Forests (RF)**

**Support Vector Machine with Gaussian Kernel (GSVM)**

**eXtreme Gradient Boosting (XGB)**

**Stratified Logistic Regression All Features**

**Stratified Logistic Regression Top 3 Features**